In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)

df_full = pd.read_pickle('data/candlestick_15m_ETHBTC_2107261130.pkl')  # This contains all the feature columns
df_full.set_index('open_time', inplace=True)
df_full.sort_index(inplace=True)
df_full.tail()

,pair_id,close_time,dow,tod,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,open,high,low,close,ma14,ma30,ma90,atr,atr_diff,atr_ma14,rsi,rsi_diff,rsi_ma14,trend_up,trend_up3,trend_up14,trend_up30,cs_ss,cs_ssr,cs_hm,cs_hmr,cs_brh,cs_buh,cs_ebu,cs_ebr,open_1,open_2,open_3,open_4,open_5,open_6,open_7,open_8,open_9,open_10,open_11,open_12,open_13,open_14,high_1,high_2,high_3,high_4,high_5,high_6,high_7,high_8,high_9,high_10,high_11,high_12,high_13,high_14,low_1,low_2,low_3,low_4,low_5,low_6,low_7,low_8,low_9,low_10,low_11,low_12,low_13,low_14,close_1,close_2,close_3,close_4,close_5,close_6,close_7,close_8,close_9,close_10,close_11,close_12,close_13,close_14,ma14_1,ma14_2,ma14_3,ma14_4,ma14_5,ma14_6,ma14_7,ma14_8,ma14_9,ma14_10,ma14_11,ma14_12,ma14_13,ma14_14,ma30_1,ma30_2,ma30_3,ma30_4,ma30_5,ma30_6,ma30_7,ma30_8,ma30_9,ma30_10,ma30_11,ma30_12,ma30_13,ma30_14,ma90_1,ma90_2,ma90_3,ma90_4,ma90_5,ma90_6,ma90_7,ma90_8,ma90_9,ma90_10,ma90_11,ma90_12,ma90_13,ma90_14,atr_1,atr_2,atr_3,atr_4,atr_5,atr_6,atr_7,atr_8,atr_9,atr_10,atr_11,atr_12,atr_13,atr_14,atr_diff_1,atr_diff_2,atr_diff_3,atr_diff_4,atr_diff_5,atr_diff_6,atr_diff_7,atr_diff_8,atr_diff_9,atr_diff_10,atr_diff_11,atr_diff_12,atr_diff_13,atr_diff_14,atr_ma14_1,atr_ma14_2,atr_ma14_3,atr_ma14_4,atr_ma14_5,atr_ma14_6,atr_ma14_7,atr_ma14_8,atr_ma14_9,atr_ma14_10,atr_ma14_11,atr_ma14_12,atr_ma14_13,atr_ma14_14,rsi_1,rsi_2,rsi_3,rsi_4,rsi_5,rsi_6,rsi_7,rsi_8,rsi_9,rsi_10,rsi_11,rsi_12,rsi_13,rsi_14,rsi_diff_1,rsi_diff_2,rsi_diff_3,rsi_diff_4,rsi_diff_5,rsi_diff_6,rsi_diff_7,rsi_diff_8,rsi_diff_9,rsi_diff_10,rsi_diff_11,rsi_diff_12,rsi_diff_13,rsi_diff_14,rsi_ma14_1,rsi_ma14_2,rsi_ma14_3,rsi_ma14_4,rsi_ma14_5,rsi_ma14_6,rsi_ma14_7,rsi_ma14_8,rsi_ma14_9,rsi_ma14_10,rsi_ma14_11,rsi_ma14_12,rsi_ma14_13,rsi_ma14_14,trend_up_1,trend_up_2,trend_up_3,trend_up_4,trend_up_5,trend_up_6,trend_up_7,trend_up_8,trend_up_9,trend_up_10,trend_up_11,trend_up_12,trend_up_13,trend_up_14,trend_up3_1,trend_up3_2,trend_up3_3,trend_up3_4,trend_up3_5,trend_up3_6,trend_up3_7,trend_up3_8,trend_up3_9,trend_up3_10,trend_up3_11,trend_up3_12,trend_up3_13,trend_up3_14,trend_up14_1,trend_up14_2,trend_up14_3,trend_up14_4,trend_up14_5,trend_up14_6,trend_up14_7,trend_up14_8,trend_up14_9,trend_up14_10,trend_up14_11,trend_up14_12,trend_up14_13,trend_up14_14,trend_up30_1,trend_up30_2,trend_up30_3,trend_up30_4,trend_up30_5,trend_up30_6,trend_up30_7,trend_up30_8,trend_up30_9,trend_up30_10,trend_up30_11,trend_up30_12,trend_up30_13,trend_up30_14,cs_ss_1,cs_ss_2,cs_ss_3,cs_ss_4,cs_ss_5,cs_ss_6,cs_ss_7,cs_ss_8,cs_ss_9,cs_ss_10,cs_ss_11,cs_ss_12,cs_ss_13,cs_ss_14,cs_ssr_1,cs_ssr_2,cs_ssr_3,cs_ssr_4,cs_ssr_5,cs_ssr_6,cs_ssr_7,cs_ssr_8,cs_ssr_9,cs_ssr_10,cs_ssr_11,cs_ssr_12,cs_ssr_13,cs_ssr_14,cs_hm_1,cs_hm_2,cs_hm_3,cs_hm_4,cs_hm_5,cs_hm_6,cs_hm_7,cs_hm_8,cs_hm_9,cs_hm_10,cs_hm_11,cs_hm_12,cs_hm_13,cs_hm_14,cs_hmr_1,cs_hmr_2,cs_hmr_3,cs_hmr_4,cs_hmr_5,cs_hmr_6,cs_hmr_7,cs_hmr_8,cs_hmr_9,cs_hmr_10,cs_hmr_11,cs_hmr_12,cs_hmr_13,cs_hmr_14,cs_brh_1,cs_brh_2,cs_brh_3,cs_brh_4,cs_brh_5,cs_brh_6,cs_brh_7,cs_brh_8,cs_brh_9,cs_brh_10,cs_brh_11,cs_brh_12,cs_brh_13,cs_brh_14,cs_buh_1,cs_buh_2,cs_buh_3,cs_buh_4,cs_buh_5,cs_buh_6,cs_buh_7,cs_buh_8,cs_buh_9,cs_buh_10,cs_buh_11,cs_buh_12,cs_buh_13,cs_buh_14,cs_ebu_1,cs_ebu_2,cs_ebu_3,cs_ebu_4,cs_ebu_5,cs_ebu_6,cs_ebu_7,cs_ebu_8,cs_ebu_9,cs_ebu_10,cs_ebu_11,cs_ebu_12,cs_ebu_13,cs_ebu_14,cs_ebr_1,cs_ebr_2,cs_ebr_3,cs_ebr_4,cs_ebr_5,cs_ebr_6,cs_ebr_7,cs_ebr_8,cs_ebr_9,cs_ebr_10,cs_ebr_11,cs_ebr_12,cs_ebr_13,cs_ebr_14
open_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-07-26 00:00:00,0,2021-07-26 00:14:59,1

In [2]:
df_col_full = set(df_full.columns)
df_col_diff = df_col_full.difference(set(['open','high','low','close']))

In [3]:
df = df_full[['open','high','low','close']].copy()

In [4]:
def add_lookback_trades(df, levels=1, cols=['open','high','low','close']):
    '''
    Adds [cols] columns for previous records to each row. New columns are
    suffixed with the record backstep. For example: open_1 means the open
    price 1 record back.
    '''
    new_df = df.copy()
    for l in range(1,levels+1):
        new_cols={c:f'{c}_{l}' for c in cols}
        new_df = new_df.merge(df[cols].shift(l).bfill().rename(columns=new_cols),
                              how='left', left_index=True, right_index=True)
    return new_df

def add_lookforward_trades(df, levels=1, cols=['open','high','low','close']):
    '''
    Adds [cols] columns for future records to each row. New columns are
    suffixed with the record forwardstep. For example: open_1 means the open
    price 1 record forward.
    '''
    new_df = df.copy()
    for l in range(1,levels+1):
        new_cols={c:f'{c}_{l}' for c in cols}
        new_df = new_df.merge(df[cols].shift(-l).ffill().rename(columns=new_cols),
                              how='left', left_index=True, right_index=True)
    return new_df

def calculate_atr(df, period=14):
    '''
    Calculate the Average True Range (ATR)
    '''
    df2=df[['high','low','close']].copy()
    df2=add_lookback_trades(df2,1,['high','low','close'])
    df2['tr_calc'] = np.max([df2.high, df2.close_1], axis=0) - np.min([df2.low, df2.close_1], axis=0)
    return df2.tr_calc.rolling(period, min_periods=1).mean()

In [5]:
def keep_only_first(y, keep=2):
    '''
    Keep only the first X "True" values in a sequence.
    For example: T,F,F,T,T,T,F,T,F  and keep=2
                 T,F,F,T,T,F,F,T,F
                           *
    '''
    boolean_out=False
    if y.dtype == bool:
        boolean_out=True
    new_y = y.copy().astype(int)
    
    prev_ones=0
    for r in new_y.iteritems():
        if r[1]==0 and prev_ones>0:
            prev_ones=0
        elif r[1]==1 and prev_ones<keep:
            prev_ones+=1
        elif r[1]==1:
            new_y.at[r[0]]=0
    
    if boolean_out:
        new_y=new_y.astype(bool)
    
    return new_y

def to_buy(x, reverse=False):
    '''
    If first candle to pass high/low column is the target,
    mark te row "True" for buying
    '''
    high_col = x.pass_high_col
    high_col_num = int(high_col[high_col.find('_')+1:])
    low_col = x.pass_low_col
    low_col_num = int(low_col[low_col.find('_')+1:])
    
    if not reverse:
        if high_col_num < low_col_num and high_col_num > 1:
            return True
        elif low_col_num == 1:
            if not x[low_col] and x[high_col]:
                return True
            else:
                return False
        else:
            return False
    else:
        if low_col_num < high_col_num and low_col_num > 1:
            return True
        elif high_col_num == 1:
            if not x[high_col] and x[low_col]:
                return True
            else:
                return False
        else:
            return False
            
def build_Xy(df, df_full, window=14, threshold_ratio=(0.04,0.02), use_atr=True,
             atr_ratio=(2,1), reverse=False, debug=False):
    '''
    Build the X,y datasets
    
    Parameters
    ----------
    window : int
        number of records to look into the future to determine if this was a buy
    
    threshold_ratio : tuple(float,float)
        The high/low percentage to calculate target/stop-loss. Ignore if use_atr is True.
    
    use_atr : boolean
        Use the ATR to calculate stop-loss
    
    atr_ratio : tuple(int,int)
        The high/low multiplier for ATR to calculate target/stop-loss. Only used when
        use_atr is True.
    
    reverse : boolean
        buying the other currency thus reverse logic
    
    debug : boolean
        show messages
    
    '''
    if use_atr==False and (not type(threshold_ratio) == tuple or len(threshold_ratio) != 2):
        raise Exception("Parameter 'threshold_ratio' must be a tuple of size 2")
    if use_atr==True and (not type(atr_ratio) == tuple or len(atr_ratio) != 2):
        raise Exception("Parameter 'atr_ratio' must be a tuple of size 2")
    
    df2 = df.copy()
    
    print(' Build: Adding look-forward trades') if debug else None
    df2 = add_lookforward_trades(df2, levels=window, cols=['high','low'])
    
    high_cols = [f'high_{i}' for i in range(1,window+1)]
    low_cols = [f'low_{i}' for i in range(1,window+1)]
    
    # Calculate target and stop-loss
    print(' Build: Calculating target and stop-loss') if debug else None
    if not reverse:
        if use_atr:
            df2['atr'] = calculate_atr(df2)
            df2['low_pip'] = (df2.close-df2.low)/df2.close
            df2['stop_loss'] = df2.close-df2.close*(df2.atr*atr_ratio[1] + df2.low_pip)
            df2['target'] = df2.close+df2.close*(df2.atr*atr_ratio[0])
        else:
            df2['stop_loss'] = df2.close-df2.close*threshold_ratio[1]
            df2['target'] = df2.close+df2.close*threshold_ratio[0]
    else:  # reverse
        if use_atr:
            df2['atr'] = calculate_atr(df2)
            df2['high_pip'] = (df2.high-df2.close)/df2.close
            df2['stop_loss'] = df2.close+df2.close*(df2.atr*atr_ratio[1] + df2.high_pip)
            df2['target'] = df2.close-df2.close*(df2.atr*atr_ratio[0])
        else:
            df2['stop_loss'] = df2.close+df2.close*threshold_ratio[1]
            df2['target'] = df2.close-df2.close*threshold_ratio[0]
            
    # Determine if price crossed the target and/or stop-loss
    print(' Build: Determining break points') if debug else None
    for h,l in zip(high_cols, low_cols):
        if not reverse:
            df2[h] = df2[h]>=df2.target
            df2[l] = df2[l]<=df2.stop_loss
        else:  # reverse
            df2[l] = df2[l]<=df2.target
            df2[h] = df2[h]>=df2.stop_loss
    
    # Identify the first column that pass the target/stop-loss
    # WARNING: this is assuming idxmax() will continue to
    # return the FIRST column with a True value.
    print(' Build: Identifying target/stop-loss columns') if debug else None
    df2['pass_high_col'] = df2[high_cols].idxmax(axis=1)
    df2['pass_low_col'] = df2[low_cols].idxmax(axis=1)
    
    print(' Build: Determining buy records') if debug else None
    df2['buy'] = df2.apply(to_buy,reverse=reverse,axis=1)
    
    # Build X
    X = df.merge(df_full, how='left', left_index=True, right_index=True)
    
    print(' Build: Dropping Unnecessary columns') if debug else None
    columns_to_drop = ['pair_id','close_time']
    X.drop(columns=columns_to_drop, inplace=True)
    
    print(' Build: Converting booleans to ints') if debug else None
    for col in X.columns:
        if X[col].dtype == bool:
            X[col] = X[col].astype(int)
    y = df2[['buy']].astype(int)
    
    #print(' Build: Trimming buy sequence to 2')
    #df2.buy = keep_only_first(df2.buy, keep=2)
    
    return X, y, df2

window=14
%time X, y, raw = build_Xy(df, df_full[df_col_diff], window=window, use_atr=True, atr_ratio=(20,5), reverse=False, debug=True)

 Build: Adding look-forward trades
 Build: Calculating target and stop-loss
 Build: Determining break points
 Build: Identifying target/stop-loss columns
 Build: Determining buy records
 Build: Dropping Unnecessary columns
 Build: Converting booleans to ints
Wall time: 3.68 s


In [6]:
suffix='20210726'
X.to_pickle(f'data/X_{suffix}.pkl')
y.to_pickle(f'data/y_{suffix}.pkl')